In [16]:
# Dependencies
from census import Census
import hvplot.pandas
import pandas as pd
import requests
import time
from scipy.stats import linregress
from matplotlib import pyplot as plt
import wbdata

# Turn off warning messages
import warnings
warnings.filterwarnings("ignore")





In [17]:
# Importing data from Stats Canada
csv_url = "Resources/owid-covid-data.csv"
covid_data = pd.read_csv(csv_url)
covid_data.columns


Index(['iso_code', 'continent', 'location', 'date', 'total_cases', 'new_cases',
       'new_cases_smoothed', 'total_deaths', 'new_deaths',
       'new_deaths_smoothed', 'total_cases_per_million',
       'new_cases_per_million', 'new_cases_smoothed_per_million',
       'total_deaths_per_million', 'new_deaths_per_million',
       'new_deaths_smoothed_per_million', 'reproduction_rate', 'icu_patients',
       'icu_patients_per_million', 'hosp_patients',
       'hosp_patients_per_million', 'weekly_icu_admissions',
       'weekly_icu_admissions_per_million', 'weekly_hosp_admissions',
       'weekly_hosp_admissions_per_million', 'total_tests', 'new_tests',
       'total_tests_per_thousand', 'new_tests_per_thousand',
       'new_tests_smoothed', 'new_tests_smoothed_per_thousand',
       'positive_rate', 'tests_per_case', 'tests_units', 'total_vaccinations',
       'people_vaccinated', 'people_fully_vaccinated', 'total_boosters',
       'new_vaccinations', 'new_vaccinations_smoothed',
       't

In [18]:
# Filtering DataFrame by columns of interest
clean_covid_data = covid_data.loc[:, ['iso_code', 'location', 'date', 'total_cases', 'new_cases', 'total_deaths', 'new_deaths', 'total_cases_per_million', 'new_cases_per_million', 'total_deaths_per_million', 'new_deaths_per_million',
                                      'icu_patients',
                                      'icu_patients_per_million', 'hosp_patients_per_million', 'new_tests_per_thousand',
                                      'positive_rate', 'people_vaccinated_per_hundred',
                                      'people_fully_vaccinated_per_hundred', 'stringency_index',
                                      'population_density', 'median_age',
                                      'gdp_per_capita', 'extreme_poverty', 'human_development_index', 'population']]

# Converting NaN to 0
clean_covid_data = clean_covid_data.fillna(0)

clean_covid_data.head(10)


iso_code     location        date  total_cases  new_cases  total_deaths  \
0      AFG  Afghanistan  2020-02-24          5.0        5.0           0.0   
1      AFG  Afghanistan  2020-02-25          5.0        0.0           0.0   
2      AFG  Afghanistan  2020-02-26          5.0        0.0           0.0   
3      AFG  Afghanistan  2020-02-27          5.0        0.0           0.0   
4      AFG  Afghanistan  2020-02-28          5.0        0.0           0.0   
5      AFG  Afghanistan  2020-02-29          5.0        0.0           0.0   
6      AFG  Afghanistan  2020-03-01          5.0        0.0           0.0   
7      AFG  Afghanistan  2020-03-02          5.0        0.0           0.0   
8      AFG  Afghanistan  2020-03-03          5.0        0.0           0.0   
9      AFG  Afghanistan  2020-03-04          5.0        0.0           0.0   

   new_deaths  total_cases_per_million  new_cases_per_million  \
0         0.0                    0.122                  0.122   
1         0.0                    0.122                  0.000   
2         0.0                    0.122                  0.000   
3         0.0                    0.122                  0.000   
4         0.0                    0.122                  0.000   
5         0.0                    0.122                  0.000   
6         0.0                    0.122                  0.000   
7         0.0                    0.122                  0.000   
8         0.0                    0.122                  0.000   
9         0.0                    0.122                  0.000   

   total_deaths_per_million  ...  positive_rate  \
0                       0.0  ...            0.0   
1                       0.0  ...            0.0   
2                       0.0  ...            0.0   
3                       0.0  ...            0.0   
4                       0.0  ...            0.0   
5                       0.0  ...            0.0   
6                       0.0  ...            0.0   
7                       0.0  ...            0.0   
8                       0.0  ...            0.0   
9                       0.0  ...            0.0   

   people_vaccinated_per_hundred  people_fully_vaccinated_per_hundred  \
0                            0.0                                  0.0   
1                            0.0                                  0.0   
2                            0.0                                  0.0   
3                            0.0                                  0.0   
4                            0.0                                  0.0   
5                            0.0                                  0.0   
6                            0.0                                  0.0   
7                            0.0                                  0.0   
8                            0.0                                  0.0   
9                            0.0                                  0.0   

   stringency_index  population_density  median_age  gdp_per_capita  \
0              8.33              54.422        18.6        1803.987   
1              8.33              54.422        18.6        1803.987   
2              8.33              54.422        18.6        1803.987   
3              8.33              54.422        18.6        1803.987   
4              8.33              54.422        18.6        1803.987   
5              8.33              54.422        18.6        1803.987   
6             27.78              54.422        18.6        1803.987   
7             27.78              54.422        18.6        1803.987   
8             27.78              54.422        18.6        1803.987   
9             27.78              54.422        18.6        1803.987   

   extreme_poverty  human_development_index  population  
0              0.0                    0.511  41128772.0  
1              0.0                    0.511  41128772.0  
2              0.0                    0.511  41128772.0  
3              0.0                    0.511  41128772.0  


In [ ]:

#Creating a new column for Month-Year
clean_covid_data.insert(3,"month_year",pd.to_datetime(clean_covid_data['date']).dt.to_period('M') )


clean_covid_data_no_date = clean_covid_data.drop('date',axis = 1)

clean_covid_data_no_date


In [ ]:
#Creating a DataFrame grouping by Month-Year and Country and adding the columns for sums and average

df_sum = clean_covid_data_no_date.groupby(['location','month_year'])[sum_columns_list].sum()



df_sum.rename(columns={"new_cases":"sum_new_cases",'new_deaths':'sum_new_deaths', 'new_cases_per_million': 'sum_new_cases_per_million'\
                        , 'new_deaths_per_million': 'sum_new_deaths_per_million' })

# print(df_sum)                        

df_mean = clean_covid_data_no_date.groupby(['location','month_year'])[mean_columns_list].mean()


df_mean.rename(columns={'icu_patients': 'average_icu_patients',
                    'icu_patients_per_million': 'average_icu_patients_per_million','hosp_patients_per_million':'average_hosp_patients_per_million'})


# print(df_mean)

df_clean_data = pd.merge(df_sum,df_mean, on =['location','month_year'])
df_clean_data



In [ ]:
#Create a DataFrame for columns that shouldnt be added nor mean (Static data e.g. population)

totals_list=['stringency_index','population_density', 'median_age',
                                      'gdp_per_capita', 'extreme_poverty', 'human_development_index', 'population']

df_totals = clean_covid_data_no_date.groupby(['location'])[totals_list].mean().astype(int)
# df_totals.style.set_precision(0)

df_totals



#John Will do this:



In [ ]:
wbdata.getGdpData()

Gus will do this